In [2]:
pwd

'C:\\Users\\hamza\\Programs\\Road_Traffic_Monitoring_AI_Vision_Based_SMART_sensor\\Notebooks'

In [3]:
cd ..

C:\Users\hamza\Programs\Road_Traffic_Monitoring_AI_Vision_Based_SMART_sensor


C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import time

#####################################################################################################################################

import cv2
import numpy as np
import torch

#####################################################################################################################################

from framegrabber.frame_grabber import FrameGrabber

from detection.ultralytics_detectors import UltralyticsDetector

from tracking.track import Tracker
from tracking.track_config_loader import TrackerConfigLoader
from tracking.track_visualizer import TrackVisualizer

#####################################################################################################################################

my_model = UltralyticsDetector("yolov8n.pt" , conf=0.1)         # "rtdetr-l.pt"  , "yolov8n.pt"
coco_vehicles = [1, 2, 3, 5, 7]

Ultralytics 8.3.196  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
Successfully yolov8n.pt model is initialized and warmedup !


In [6]:
tracker_load = TrackerConfigLoader(classes=coco_vehicles)
trackers = tracker_load.load_tracker_areas()

trackers_list = [trackers[1]] + trackers[0]

area_info = tracker_load.load_area_info()
track_vis = TrackVisualizer(areas = area_info)
trackers_list

In [7]:
source = "vid1.mp4"                   # 0 "kech.mp4"
stride = 1
stride_method = "periodic_stride"             # "burst_stride", "periodic_stride", "random_sampling"

frame_grabber = FrameGrabber(source,
                             stride=stride,
                             stride_method=stride_method,
                             window_size=20,
                             queue_maxsize=10,
                             grabber_mode = "queue",
                             fallback_fps=30,
                             allow_dynamic_resolution=False)


if not frame_grabber.open():
    raise RuntimeError("Failed to open source")
# ensure window exists (main thread)
cv2.namedWindow('BoXMOT + ultralytics', cv2.WINDOW_NORMAL)
if frame_grabber._grabber_mode=="queue":
# start producer
    frame_grabber.start()


try:
    with torch.inference_mode():
        while True:
            # try to get a frame but don't block forever
            frame = frame_grabber.get_frame(timeout=0.1)  # <-- short timeout keeps loop responsive
                
            if frame is not None:
                
                print(f"frame_grabber index: {frame.read_idx}")
                # run detection/tracking only if we have a frame
                ready_to_track_array = my_model.detect_to_track(frame.data)
                
                g_track = trackers_list[0].update(ready_to_track_array, frame.data)
                # roi_track_1 = trackers_list[1].update(ready_to_track_array, frame.data)

                track_plot = track_vis.annotate(frame.data , g_track)
                
                # mark processed & show
                frame_grabber.mark_processed(frame)

                cv2.imshow('BoXMOT + ultralytics', track_plot)
                    
            else:
                # no frame this iteration (timeout), you may choose to display a placeholder
                # or simply continue — but still poll for key events below
                pass

            # ALWAYS poll keyboard events so 'q' is detected even when no frame was available
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                # stop producer and break loop
                if frame_grabber._grabber_mode=="queue":
                    frame_grabber.stop(wait=True)
                break

            # optional: also break when producer finished (sentinel)
            if frame_grabber._grabber_mode=="queue":
                if frame is None and frame_grabber._stop_event.is_set():
                    break
finally:
    frame_grabber.release()
    cv2.destroyAllWindows()

frame_grabber index: 0
frame_grabber index: 1
frame_grabber index: 2
frame_grabber index: 3
frame_grabber index: 4
frame_grabber index: 5
frame_grabber index: 6
frame_grabber index: 7
frame_grabber index: 8
frame_grabber index: 9
frame_grabber index: 10
frame_grabber index: 11
frame_grabber index: 12
frame_grabber index: 13
frame_grabber index: 14
frame_grabber index: 15
frame_grabber index: 16
frame_grabber index: 17
frame_grabber index: 18
frame_grabber index: 19
frame_grabber index: 20
frame_grabber index: 21
frame_grabber index: 22
frame_grabber index: 23
frame_grabber index: 24
frame_grabber index: 25
frame_grabber index: 26
frame_grabber index: 27
frame_grabber index: 28
frame_grabber index: 29
frame_grabber index: 30
frame_grabber index: 31
frame_grabber index: 32
frame_grabber index: 33
frame_grabber index: 34
frame_grabber index: 35
frame_grabber index: 36
frame_grabber index: 37
frame_grabber index: 38
frame_grabber index: 39
frame_grabber index: 40
frame_grabber index: 41
fr